In [1]:
%pip install tensorflow==2.13.0 opencv-python mediapipe sklearn matplotlib 
%pip install keras  

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential # És una API del keras que permet crear els models/XNs simplemt definint quines capes vols i algunes de les seves caracteristiques.
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard # Eina per visualitzar i monitoritzar informació relacionada amb el procés d'entrenament del model


In [6]:
mp_holistic = mp.solutions.holistic # Asignem a la variable mp_holistic el model de MediaPipe anomenat Holistic, l'encarregat de detectar la postura i fer-ne el seu seguiment, que hem agafat del modul mp.solutions (on hi ha altres models de CV per altres tasques)
mp_drawing = mp.solutions.drawing_utils # Ve a ser una eina (que conté altres eines) que utilitzarem per "dibuixar" o visualitzar de forma gràfica els resultats del model a sobre de la imatge 

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Conversió de color: de BGR a RGB (OpenCV proporciona la imatge en BGR i el model de MP necestila la entrada en RGB)
    image.flags.writeable = False                  # Fem que la imatge no sigui "escribible" (per evitar que es modifiqui mentres la procesa el model)
    results = model.process(image)                 # Passem la imatge al model perque la procesi i guardi els resultats en la variable
    image.flags.writeable = True                   # Desfem operació simètrica
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Conversio color RGB a BGR 
    return image, results

def draw_styled_landmarks(image, results):
    # Dibuixar connecions de la cara
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Dibuixar connecions de la postura
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Dibuixar connecions de la mà esquerra
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Dibuixar connecions de la mà dreta 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results): 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) 




# Els noms dels gests que volem detectar (les etiquetes/labels)
actions = np.array(['estimar', 'adeu', 'gracies'])

# Numero de videos (seqüències) que utilitzarem per l'entrenament
no_sequences = 30

# Numero de frames per cada video (llargada)
sequence_length = 30

# Definim el model i les seves caracteristiques
model = Sequential() # Creem un model sequencial
#Primera capa LSTM del model. 64 neurones o unitats. Funció d'activació ReLU (la millor per les LSTM). Definim que cada input (que és un video/sequència) tindrà una longitit de 30 frames amb 1662 caracteristiques per frame
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) # La seguent capa ja no és LSTM, per tant no li passem la sequencia sencera. Posant false significa que li pasarem cada output dins la sequencia un per un sense parar.
# Aqui posem capes de tipus dense (que ve a ser el tipus de capa per defecte en general, una feedforward fully-connected)
# Aquestes aniràn bé perquè busquin relacions més complexes i abstractes a mida que avançen (a la vegada reduint el nombre de neurones pel mateix motiu fer el pas a la capa de sortida més suau)
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # El numero d'unitats de sortida serà igual al numero d'accions que hagim establert. Funció d'activaciño de softmax per clasificar els resultats, donant la probabilitat (en tant per 1) que aquest resultats siguin d'una clase o d'una altra.

model.load_weights('my_model_weights.h5')
# Així podem recuperar el model entrenat pel seu ús en altres dispositius o en el mateix (al tancar la Jupyter notebook per exemple es perden els pesos) sense haver de repetir tots els passos anteriors.

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

sequence = [] # Aquí anirem guardant els frames fins que tinguem 30, doncs ho passarem pel model perque faci les prediccions necesaries
sentence = [] # El historial de les paraules que s'hagin dit. Seria util en el cas que tinguessim una gran base de dades amb gestos i poguessim traduir frases
predictions = []
threshold = 0.5 # El minim de confiança que ha de tenir el model perque el resultat es mostri en pantalla

cap = cv2.VideoCapture(0)
# definir el model pels keypoints 
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():

        # RLlegir el feed
        ret, frame = cap.read()

        # Fer les deteccions dels keyframes
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Dibuixar els resultats
        draw_styled_landmarks(image, results)
        
        # 2. Predicció
        keypoints = extract_keypoints(results) # Agafem els resultats del holistic (els resultats del fotograma on estiguem)
        sequence.append(keypoints) # Els fiquem al final de sequence
        sequence = sequence[-30:] # Agafem nomes els ultims 30 valors que hagim guardat allà (o sigui els keypoints dels ultims 30 fotogrames) 
        
        if len(sequence) == 30: # Si tenim 30 frames de resultats, fer el seguent
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # Haguessim posat només que el model fes la predicció directament dels valors emmagatzemats a sequence però la forma del array que tenim no és la que el model demana. (li estem donant un array amb dos x (30, 1662) pero espera un amb tres (numero de la sequencia, 30, 1662))
            print(actions[np.argmax(res)]) # Opcional
            predictions.append(np.argmax(res))
            
            
        #3. Visualització
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: # Agafem la predicció amb més probabilitats de ser la correcta i comparem el tant per 1 de possibilitats que ho sigui amb el nostre minim establert a threshold. Si és superior a aquest, el posarem en pantalla.
                    
                    if len(sentence) > 0: # Mirem si ja hem predit alguna paraula i està en la frase
                        if actions[np.argmax(res)] != sentence[-1]: # Si ja tenim alguna paraula a la frase el que farem és asegurar-nos que la nova paraula que hem predit no sigui la ultima que hi ha a la frase.
                            sentence.append(actions[np.argmax(res)]) # Si la paraula nova no és la ultima que apareix a la frase (és una diferent), l'afegim a aquesta. Això ho fem perque, com fa les prediccions molt rapid, no volem que aparegui una paraula varies vegades quan nomes hem fet el gest una vegada. 
                    else: # Si no tenim cap paraula a la frase, afegim la paraula que acabem de predir
                        sentence.append(actions[np.argmax(res)])

            # Si ja tenim 5 paraules a la frase i afegim una nova, simplement agafem les ultimes cuatre i la nova i oblidem la ultima per no saturar la pantalla
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Visualitzar les probabilitats de les paraules
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1) # Punt superior esquerra del rectange, punt inferior dret, color
        cv2.putText(image, ' '.join(sentence), (3,30), # Posem la frase a la pantalla
                       cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        # Mostrar en pantalla
        cv2.imshow('OpenCV Feed', image)

        # Tancar
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti